In [ ]:
# ⏳ Anti-Idle (mantém Colab ativo)
import time
while True:
    time.sleep(60)

In [ ]:
# 📂 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ⬇️ Clonar WebUI
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/stable-diffusion-webui

In [ ]:
# 🔗 Linkar pasta de LoRAs
!mkdir -p /content/stable-diffusion-webui/models/Lora
!ln -s /content/drive/MyDrive/SD_LoRAs /content/stable-diffusion-webui/models/Lora

In [ ]:
# ▶️ Rodar WebUI em modo CPU (sem GPU)
%cd /content/stable-diffusion-webui
!COMMANDLINE_ARGS="--share --disable-safe-unpickle --skip-torch-cuda-test --precision full --no-half" \
  REQS_FILE="requirements.txt" \
  python launch.py